In [ ]:
!wget https://dlcdn.apache.org/spark/spark-3.4.0/spark-3.4.0-bin-hadoop3.tgz
!ls
!tar -xvzf spark-3.4.0-bin-hadoop3.tgz
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.0-bin-hadoop3"
!pip install findspark
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [4]:
!wget https://raw.githubusercontent.com/futurexskill/bigdata/master/retailstore.csv

--2023-06-15 17:43:25--  https://raw.githubusercontent.com/futurexskill/bigdata/master/retailstore.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 279 [text/plain]
Saving to: ‘retailstore.csv’

retailstore.csv     100%[===================>]     279  --.-KB/s    in 0s      

2023-06-15 17:43:26 (10.9 MB/s) - ‘retailstore.csv’ saved [279/279]



In [5]:
spark = SparkSession.builder.master("local[*]").appName("df example").enableHiveSupport().getOrCreate()

In [6]:
customer_df = spark.read.csv("retailstore.csv",header=True)

In [7]:
type(customer_df)

pyspark.sql.dataframe.DataFrame

In [8]:
customer_df.show()

+----+------+------+-------+---------+
| Age|Salary|Gender|Country|Purchased|
+----+------+------+-------+---------+
|  18| 20000|  Male|Germany|        N|
|  19| 22000|Female| France|        N|
|  20| 24000|Female|England|        N|
|  21|  null|  Male|England|        N|
|  22| 50000|  Male| France|        Y|
|  23| 35000|Female|England|        N|
|  24|  null|  Male|Germany|        N|
|  25| 32000|Female| France|        Y|
|null| 35000|  Male|Germany|        N|
|  27| 37000|Female| France|        N|
+----+------+------+-------+---------+



In [9]:
customer_df.select("country").show()

+-------+
|country|
+-------+
|Germany|
| France|
|England|
|England|
| France|
|England|
|Germany|
| France|
|Germany|
| France|
+-------+



In [10]:
customer_df.groupBy("gender").count().show()

+------+-----+
|gender|count|
+------+-----+
|Female|    5|
|  Male|    5|
+------+-----+



In [11]:
customer_df.groupBy("country").count().show()

+-------+-----+
|country|count|
+-------+-----+
|Germany|    3|
| France|    4|
|England|    3|
+-------+-----+



In [12]:
customer_df.createOrReplaceTempView("customer")

In [13]:
results =spark.sql("select * from customer")

In [14]:
results.show()

+----+------+------+-------+---------+
| Age|Salary|Gender|Country|Purchased|
+----+------+------+-------+---------+
|  18| 20000|  Male|Germany|        N|
|  19| 22000|Female| France|        N|
|  20| 24000|Female|England|        N|
|  21|  null|  Male|England|        N|
|  22| 50000|  Male| France|        Y|
|  23| 35000|Female|England|        N|
|  24|  null|  Male|Germany|        N|
|  25| 32000|Female| France|        Y|
|null| 35000|  Male|Germany|        N|
|  27| 37000|Female| France|        N|
+----+------+------+-------+---------+



In [15]:
type(results)

pyspark.sql.dataframe.DataFrame

In [16]:
filter_results = spark.sql("select * from customer where salary > 30000").show()

+----+------+------+-------+---------+
| Age|Salary|Gender|Country|Purchased|
+----+------+------+-------+---------+
|  22| 50000|  Male| France|        Y|
|  23| 35000|Female|England|        N|
|  25| 32000|Female| France|        Y|
|null| 35000|  Male|Germany|        N|
|  27| 37000|Female| France|        N|
+----+------+------+-------+---------+



In [17]:
customer_df.filter("salary > 30000").show()

+----+------+------+-------+---------+
| Age|Salary|Gender|Country|Purchased|
+----+------+------+-------+---------+
|  22| 50000|  Male| France|        Y|
|  23| 35000|Female|England|        N|
|  25| 32000|Female| France|        Y|
|null| 35000|  Male|Germany|        N|
|  27| 37000|Female| France|        N|
+----+------+------+-------+---------+



In [18]:
customer_df.groupby("country").agg({"salary":"avg","age":"max"}).show()

+-------+-----------+--------+
|country|avg(salary)|max(age)|
+-------+-----------+--------+
|England|    29500.0|      23|
| France|    35250.0|      27|
|Germany|    27500.0|      24|
+-------+-----------+--------+



In [19]:
spark.sql("select country, avg(salary), max(age)from customer group by country ").show()

+-------+-----------+--------+
|country|avg(salary)|max(age)|
+-------+-----------+--------+
|England|    29500.0|      23|
| France|    35250.0|      27|
|Germany|    27500.0|      24|
+-------+-----------+--------+



In [20]:
customer_df.select(["country","salary"]).show()

+-------+------+
|country|salary|
+-------+------+
|Germany| 20000|
| France| 22000|
|England| 24000|
|England|  null|
| France| 50000|
|England| 35000|
|Germany|  null|
| France| 32000|
|Germany| 35000|
| France| 37000|
+-------+------+



In [21]:
customer_df.withColumn("doubleSalary",customer_df["salary"]*2).show()

+----+------+------+-------+---------+------------+
| Age|Salary|Gender|Country|Purchased|doubleSalary|
+----+------+------+-------+---------+------------+
|  18| 20000|  Male|Germany|        N|     40000.0|
|  19| 22000|Female| France|        N|     44000.0|
|  20| 24000|Female|England|        N|     48000.0|
|  21|  null|  Male|England|        N|        null|
|  22| 50000|  Male| France|        Y|    100000.0|
|  23| 35000|Female|England|        N|     70000.0|
|  24|  null|  Male|Germany|        N|        null|
|  25| 32000|Female| France|        Y|     64000.0|
|null| 35000|  Male|Germany|        N|     70000.0|
|  27| 37000|Female| France|        N|     74000.0|
+----+------+------+-------+---------+------------+



In [34]:
from pyspark.sql.functions import lit
customer_df.withColumn("new_col",lit("K")).show()


+----+------+------+-------+---------+-------+
| Age|Salary|Gender|Country|Purchased|new_col|
+----+------+------+-------+---------+-------+
|  18| 20000|  Male|Germany|        N|      K|
|  19| 22000|Female| France|        N|      K|
|  20| 24000|Female|England|        N|      K|
|  21|  null|  Male|England|        N|      K|
|  22| 50000|  Male| France|        Y|      K|
|  23| 35000|Female|England|        N|      K|
|  24|  null|  Male|Germany|        N|      K|
|  25| 32000|Female| France|        Y|      K|
|null| 35000|  Male|Germany|        N|      K|
|  27| 37000|Female| France|        N|      K|
+----+------+------+-------+---------+-------+



In [22]:
customer_df.withColumnRenamed("salary", "income").show()

+----+------+------+-------+---------+
| Age|income|Gender|Country|Purchased|
+----+------+------+-------+---------+
|  18| 20000|  Male|Germany|        N|
|  19| 22000|Female| France|        N|
|  20| 24000|Female|England|        N|
|  21|  null|  Male|England|        N|
|  22| 50000|  Male| France|        Y|
|  23| 35000|Female|England|        N|
|  24|  null|  Male|Germany|        N|
|  25| 32000|Female| France|        Y|
|null| 35000|  Male|Germany|        N|
|  27| 37000|Female| France|        N|
+----+------+------+-------+---------+



In [23]:
customer_df.filter("salary > 30000").select("age").show()

+----+
| age|
+----+
|  22|
|  23|
|  25|
|null|
|  27|
+----+



In [24]:
customer_df.filter("salary > 30000").select(["age","country"]).show()

+----+-------+
| age|country|
+----+-------+
|  22| France|
|  23|England|
|  25| France|
|null|Germany|
|  27| France|
+----+-------+



In [27]:
customer_df.select("salary","age", "country").show()

+------+----+-------+
|salary| age|country|
+------+----+-------+
| 20000|  18|Germany|
| 22000|  19| France|
| 24000|  20|England|
|  null|  21|England|
| 50000|  22| France|
| 35000|  23|England|
|  null|  24|Germany|
| 32000|  25| France|
| 35000|null|Germany|
| 37000|  27| France|
+------+----+-------+



In [33]:
customer_df.filter("salary > 30000 and salary < 40000").select(["age","country"]).show()

+----+-------+
| age|country|
+----+-------+
|  23|England|
|  25| France|
|null|Germany|
|  27| France|
+----+-------+



In [29]:
from pyspark.sql.functions import countDistinct, avg, stddev

In [34]:
customer_df.select(countDistinct("country")).show()

+-----------------------+
|count(DISTINCT country)|
+-----------------------+
|                      3|
+-----------------------+



In [35]:
customer_df.select(countDistinct("country").alias("distinct countries count")).show()

+------------------------+
|distinct countries count|
+------------------------+
|                       3|
+------------------------+



In [36]:
customer_df.select(avg("salary")).show()

+-----------+
|avg(salary)|
+-----------+
|    31875.0|
+-----------+



In [37]:
customer_df.select(avg("age")).show()

+-----------------+
|         avg(age)|
+-----------------+
|22.11111111111111|
+-----------------+



In [38]:
customer_df.select(stddev("salary")).show()

+-------------------+
|stddev_samp(salary)|
+-------------------+
|  9818.895777311942|
+-------------------+



In [39]:
from pyspark.sql.functions import format_number

In [41]:
salary_std=customer_df.select(stddev("salary").alias("std"))

In [42]:
salary_std.show()

+-----------------+
|              std|
+-----------------+
|9818.895777311942|
+-----------------+



In [43]:
salary_std.select(format_number("std",2)).show()

+---------------------+
|format_number(std, 2)|
+---------------------+
|             9,818.90|
+---------------------+



In [44]:
customer_df.orderBy("salary").show()

+----+------+------+-------+---------+
| Age|Salary|Gender|Country|Purchased|
+----+------+------+-------+---------+
|  21|  null|  Male|England|        N|
|  24|  null|  Male|Germany|        N|
|  18| 20000|  Male|Germany|        N|
|  19| 22000|Female| France|        N|
|  20| 24000|Female|England|        N|
|  25| 32000|Female| France|        Y|
|  23| 35000|Female|England|        N|
|null| 35000|  Male|Germany|        N|
|  27| 37000|Female| France|        N|
|  22| 50000|  Male| France|        Y|
+----+------+------+-------+---------+



In [46]:
from pyspark.sql.functions import col

In [47]:
customer_df.orderBy(col("salary").desc()).show()

+----+------+------+-------+---------+
| Age|Salary|Gender|Country|Purchased|
+----+------+------+-------+---------+
|  22| 50000|  Male| France|        Y|
|  27| 37000|Female| France|        N|
|  23| 35000|Female|England|        N|
|null| 35000|  Male|Germany|        N|
|  25| 32000|Female| France|        Y|
|  20| 24000|Female|England|        N|
|  19| 22000|Female| France|        N|
|  18| 20000|  Male|Germany|        N|
|  21|  null|  Male|England|        N|
|  24|  null|  Male|Germany|        N|
+----+------+------+-------+---------+



In [50]:
customer_df.orderBy(col("salary").asc()).show()

+----+------+------+-------+---------+
| Age|Salary|Gender|Country|Purchased|
+----+------+------+-------+---------+
|  21|  null|  Male|England|        N|
|  24|  null|  Male|Germany|        N|
|  18| 20000|  Male|Germany|        N|
|  19| 22000|Female| France|        N|
|  20| 24000|Female|England|        N|
|  25| 32000|Female| France|        Y|
|  23| 35000|Female|England|        N|
|null| 35000|  Male|Germany|        N|
|  27| 37000|Female| France|        N|
|  22| 50000|  Male| France|        Y|
+----+------+------+-------+---------+



In [51]:
new_df = customer_df.drop("purchased")

In [52]:
new_df.show()

+----+------+------+-------+
| Age|Salary|Gender|Country|
+----+------+------+-------+
|  18| 20000|  Male|Germany|
|  19| 22000|Female| France|
|  20| 24000|Female|England|
|  21|  null|  Male|England|
|  22| 50000|  Male| France|
|  23| 35000|Female|England|
|  24|  null|  Male|Germany|
|  25| 32000|Female| France|
|null| 35000|  Male|Germany|
|  27| 37000|Female| France|
+----+------+------+-------+



In [53]:
clean_df = customer_df.na.drop()

In [54]:
clean_df.show()

+---+------+------+-------+---------+
|Age|Salary|Gender|Country|Purchased|
+---+------+------+-------+---------+
| 18| 20000|  Male|Germany|        N|
| 19| 22000|Female| France|        N|
| 20| 24000|Female|England|        N|
| 22| 50000|  Male| France|        Y|
| 23| 35000|Female|England|        N|
| 25| 32000|Female| France|        Y|
| 27| 37000|Female| France|        N|
+---+------+------+-------+---------+



In [59]:
correct_df1 = customer_df.na.fill("n/a")

In [60]:
correct_df1.show()

+---+------+------+-------+---------+
|Age|Salary|Gender|Country|Purchased|
+---+------+------+-------+---------+
| 18| 20000|  Male|Germany|        N|
| 19| 22000|Female| France|        N|
| 20| 24000|Female|England|        N|
| 21|   n/a|  Male|England|        N|
| 22| 50000|  Male| France|        Y|
| 23| 35000|Female|England|        N|
| 24|   n/a|  Male|Germany|        N|
| 25| 32000|Female| France|        Y|
|n/a| 35000|  Male|Germany|        N|
| 27| 37000|Female| France|        N|
+---+------+------+-------+---------+



In [ ]:
l